In [69]:
# Value Creation Bridge aligned to Achleitner et al. 2010; Achleitner, Braun & Puche 2015; Söffge & Braun 2017)

# === Core Value-Bridge Items: times_money and tm_unlevered (exit-row div/cap injection) ===

from pathlib import Path
import pandas as pd
import numpy as np

# ---- helper ----
def find_upwards(rel_path: Path, max_up: int = 8) -> Path:
    here = Path.cwd()
    for parent in [here, *here.parents][: max_up + 1]:
        candidate = (parent / rel_path)
        if candidate.exists():
            return candidate.resolve()
    raise FileNotFoundError(
        f"Couldn't locate '{rel_path.as_posix()}' from {here} by walking up {max_up} levels.\n"
        f"- Current working directory: {here}\n"
        f"- Checked: {[str((p / rel_path)) for p in [here, *here.parents][: max_up + 1]]}"
    )

TARGET_CSV = (find_upwards(Path("ValueCreation")) / "Data" / "working.csv")

# ---- load + checks ----
df = pd.read_csv(TARGET_CSV, dtype={"id": str, "deal_id": str})
assert df.groupby("deal_id")["id"].nunique().eq(2).all(), "Each deal_id must have exactly 2 rows."

def num(s): return pd.to_numeric(s, errors="coerce")

# Order within deal: 1 = entry (earliest), 2 = exit (latest)
df["_ref_dt"] = pd.to_datetime(df["reference_date"], errors="coerce")
df["_rank"]   = df.groupby("deal_id")["_ref_dt"].rank(method="first", ascending=True)

eq  = num(df["equity"])
der = num(df["de_ratio"])

# Entry / Exit equity
equity_entry = (df.loc[df["_rank"] == 1, ["deal_id"]]
                  .assign(equity_entry=eq[df["_rank"] == 1].values)
                  .groupby("deal_id")["equity_entry"].first())
equity_exit  = (df.loc[df["_rank"] == 2, ["deal_id"]]
                  .assign(equity_exit=eq[df["_rank"] == 2].values)
                  .groupby("deal_id")["equity_exit"].first())

# ---- deal-level inputs (exit-row dividends & capital injections) ----
div_exit = (df.loc[df["_rank"] == 2, ["deal_id"]]
              .assign(div_exit=num(df["dividends"])[df["_rank"] == 2].values)
              .groupby("deal_id")["div_exit"].first())

cap_exit = (df.loc[df["_rank"] == 2, ["deal_id"]]
              .assign(cap_exit=num(df["capital_injections"])[df["_rank"] == 2].values)
              .groupby("deal_id")["cap_exit"].first())

# Averages unchanged
der_avg = df.groupby("deal_id")["de_ratio"].apply(lambda s: num(s).mean())
cod_avg = df.groupby("deal_id")["cost_of_debt"].apply(lambda s: num(s).mean())

# ---- formulas (deal level) ----
d_equity = equity_exit - equity_entry
net_capital_gain = d_equity + div_exit + cap_exit
invested_capital = equity_entry - cap_exit

with np.errstate(divide="ignore", invalid="ignore"):
    times_money = (net_capital_gain / invested_capital).where(invested_capital != 0)

with np.errstate(divide="ignore", invalid="ignore"):
    tm_unlevered = (times_money + cod_avg * der_avg) / (1 + der_avg)

leverage_effect = times_money - tm_unlevered

# ---- broadcast to both rows ----
deal_metrics = pd.DataFrame({
    "deal_id": d_equity.index,
    "d_equity": d_equity.values,
    "net_capital_gain": net_capital_gain.values,
    "invested_capital": invested_capital.values,
    "times_money": times_money.values,
    "tm_unlevered": tm_unlevered.values,
    "leverage_effect": leverage_effect.values,
}).set_index("deal_id")

out = df.drop(columns=["_ref_dt","_rank"]).merge(
    deal_metrics, left_on="deal_id", right_index=True, how="left"
)

# ---- persist + checks ----
out.to_csv(TARGET_CSV, index=False)

chk = pd.read_csv(TARGET_CSV, dtype={"deal_id": str})
assert chk.groupby("deal_id")["id"].nunique().eq(2).all(), "Row cardinality changed."
for c in ["d_equity","net_capital_gain","invested_capital","times_money","tm_unlevered","leverage_effect"]:
    assert c in chk.columns, f"Missing column: {c}"

print("Deal-level metrics added (exit-row div/cap applied):",
      ["d_equity","net_capital_gain","invested_capital","times_money","tm_unlevered","leverage_effect"])


Deal-level metrics added (exit-row div/cap applied): ['d_equity', 'net_capital_gain', 'invested_capital', 'times_money', 'tm_unlevered', 'leverage_effect']


In [70]:
# === Absolute Value Drivers: Multiple Effect, FCF Effect, EBITDA Effect ===

from pathlib import Path
import pandas as pd
import numpy as np

# ---- helper ----
def find_upwards(rel_path: Path, max_up: int = 8) -> Path:
    here = Path.cwd()
    for parent in [here, *here.parents][: max_up + 1]:
        candidate = (parent / rel_path)
        if candidate.exists():
            return candidate.resolve()
    raise FileNotFoundError(
        f"Couldn't locate '{rel_path.as_posix()}' from {here} by walking up {max_up} levels.\n"
        f"- Current working directory: {here}\n"
        f"- Checked: {[str((p / rel_path)) for p in [here, *here.parents][: max_up + 1]]}"
    )

TARGET_CSV = (find_upwards(Path("ValueCreation")) / "Data" / "working.csv")

# ---- load + guarantees ----
df = pd.read_csv(TARGET_CSV, dtype={"id": str, "deal_id": str})
assert df.groupby("deal_id")["id"].nunique().eq(2).all(), "Each deal_id must have exactly 2 rows."

# ---- ordering within deal: entry (earliest) vs exit (latest) ----
df["_ref_dt"] = pd.to_datetime(df["reference_date"], errors="coerce")
# rank ascending: 1 = entry (earliest), 2 = exit (latest)
df["_rank"] = df.groupby("deal_id")["_ref_dt"].rank(method="first", ascending=True)

# convenience
def num(s): return pd.to_numeric(s, errors="coerce")

xe  = num(df["xebitda"])
eb  = num(df["ebitda"])
rv  = num(df["revenue"])
mg  = num(df["ebitda_margin"])
nd  = num(df["net_debt"])
div = num(df["dividends"])
cap = num(df["capital_injections"])

# ---- pick entry / exit values per deal ----
# entry = rank 1; exit = rank 2
def pick(series, rank_val, name):
    s = series[df["_rank"] == rank_val]
    out = df.loc[df["_rank"] == rank_val, ["deal_id"]].assign(**{name: s.values})
    return out.groupby("deal_id")[name].first()

xebitda_entry  = pick(xe, 1, "xebitda_entry")
xebitda_exit   = pick(xe, 2, "xebitda_exit")
ebitda_entry   = pick(eb, 1, "ebitda_entry")
ebitda_exit    = pick(eb, 2, "ebitda_exit")
revenue_entry  = pick(rv, 1, "revenue_entry")
revenue_exit   = pick(rv, 2, "revenue_exit")
margin_entry   = pick(mg, 1, "margin_entry")
margin_exit    = pick(mg, 2, "margin_exit")
net_debt_entry = pick(nd, 1, "net_debt_entry")
net_debt_exit  = pick(nd, 2, "net_debt_exit")
# dividends / capital_injections from EXIT row, as specified
div_exit       = pick(div, 2, "div_exit")
cap_exit       = pick(cap, 2, "cap_exit")

# sanitize non-finite drivers used in products
xebitda_entry  = xebitda_entry.where(np.isfinite(xebitda_entry))
margin_entry   = margin_entry.where(np.isfinite(margin_entry))

# ---- deltas (exit - entry), except d_debt = entry - exit (deleverage positive) ----
d_multiple = xebitda_exit - xebitda_entry
d_ebitda   = ebitda_exit  - ebitda_entry
d_revenue  = revenue_exit - revenue_entry
d_margin   = margin_exit  - margin_entry
d_debt     = net_debt_entry - net_debt_exit  # entry − exit (intentional)

# ---- effects ----
# 1) multiple_effect = ΔMultiple × EBITDA_entry
multiple_effect = d_multiple * ebitda_entry

# 2) fcf_effect = d_debt + dividends_exit + capital_injections_exit
fcf_effect = d_debt + div_exit + cap_exit

# 3) multiple_ebitda_combination_effect = ΔMultiple × ΔEBITDA
multiple_ebitda_combination_effect = d_multiple * d_ebitda

# 4) ebitda_effect = ΔEBITDA × Multiple_entry
ebitda_effect = d_ebitda * xebitda_entry

# 5) sales_effect = ΔRevenue × margin_entry × Multiple_entry
sales_effect = d_revenue * margin_entry * xebitda_entry

# 6) margin_effect = ΔMargin × revenue_entry × Multiple_entry
margin_effect = d_margin * revenue_entry * xebitda_entry

# 7) sales_margin_combination_effect = ΔRevenue × ΔMargin × Multiple_entry
sales_margin_combination_effect = d_revenue * d_margin * xebitda_entry

# ---- bundle per-deal results and broadcast to both rows ----
deal_effects = pd.DataFrame({
    "deal_id": d_multiple.index,
    "multiple_effect": multiple_effect.values,
    "fcf_effect": fcf_effect.values,
    "multiple_ebitda_combination_effect": multiple_ebitda_combination_effect.values,
    "ebitda_effect": ebitda_effect.values,
    "sales_effect": sales_effect.values,
    "margin_effect": margin_effect.values,
    "sales_margin_combination_effect": sales_margin_combination_effect.values,
}).set_index("deal_id")

out = df.drop(columns=["_ref_dt", "_rank"]).merge(
    deal_effects, left_on="deal_id", right_index=True, how="left"
)

# ---- persist + minimal checks ----
out.to_csv(TARGET_CSV, index=False)

chk = pd.read_csv(TARGET_CSV, dtype={"deal_id": str})
assert chk.groupby("deal_id")["id"].nunique().eq(2).all(), "Row cardinality changed."
for c in [
    "multiple_effect","fcf_effect","multiple_ebitda_combination_effect",
    "ebitda_effect","sales_effect","margin_effect","sales_margin_combination_effect"
]:
    assert c in chk.columns, f"Missing column: {c}"

print("Deal-level effects added:",
      ["multiple_effect","fcf_effect","multiple_ebitda_combination_effect",
       "ebitda_effect","sales_effect","margin_effect","sales_margin_combination_effect"])


Deal-level effects added: ['multiple_effect', 'fcf_effect', 'multiple_ebitda_combination_effect', 'ebitda_effect', 'sales_effect', 'margin_effect', 'sales_margin_combination_effect']


In [71]:
# === Relative Value Drivers: Multiple Effect, FCF Effect, EBITDA Effect ===

from pathlib import Path
import pandas as pd
import numpy as np

# ---- helper ----
def find_upwards(rel_path: Path, max_up: int = 8) -> Path:
    here = Path.cwd()
    for parent in [here, *here.parents][: max_up + 1]:
        candidate = (parent / rel_path)
        if candidate.exists():
            return candidate.resolve()
    raise FileNotFoundError(
        f"Couldn't locate '{rel_path.as_posix()}' from {here} by walking up {max_up} levels.\n"
        f"- Current working directory: {here}\n"
        f"- Checked: {[str((p / rel_path)) for p in [here, *here.parents][: max_up + 1]]}"
    )

TARGET_CSV = (find_upwards(Path("ValueCreation")) / "Data" / "working.csv")

# ---- load + guarantee ----
df = pd.read_csv(TARGET_CSV, dtype={"id": str, "deal_id": str})
assert df.groupby("deal_id")["id"].nunique().eq(2).all(), "Each deal_id must have exactly 2 rows."

# ---- relative effects ----
effects = [
    "multiple_effect",
    "fcf_effect",
    "multiple_ebitda_combination_effect",
    "ebitda_effect",
    "sales_effect",
    "margin_effect",
    "sales_margin_combination_effect",
]

ncg = pd.to_numeric(df["net_capital_gain"], errors="coerce")

with np.errstate(divide="ignore", invalid="ignore"):
    for e in effects:
        val = pd.to_numeric(df[e], errors="coerce")
        df[f"relative_{e}"] = (val / ncg).where(ncg != 0)

# ---- persist ----
df.to_csv(TARGET_CSV, index=False)

print("Computed relative effects for:", effects)


Computed relative effects for: ['multiple_effect', 'fcf_effect', 'multiple_ebitda_combination_effect', 'ebitda_effect', 'sales_effect', 'margin_effect', 'sales_margin_combination_effect']


In [72]:
# === QA + TM_unlevered-scaled (tmu_contrib_) driver contributions, including EBITDA sub-breakdown ===

from pathlib import Path
import pandas as pd
import numpy as np

# ---- helper ----
def find_upwards(rel_path: Path, max_up: int = 8) -> Path:
    here = Path.cwd()
    for parent in [here, *here.parents][: max_up + 1]:
        cand = parent / rel_path
        if cand.exists():
            return cand.resolve()
    raise FileNotFoundError(
        f"Couldn't locate '{rel_path.as_posix()}' from {here} by walking up {max_up} levels."
    )

TARGET_CSV = (find_upwards(Path("ValueCreation")) / "Data" / "working.csv")

# ---- load + guarantee ----
df = pd.read_csv(TARGET_CSV, dtype={"id": str, "deal_id": str})
assert df.groupby("deal_id")["id"].nunique().eq(2).all(), "Each deal_id must have exactly 2 rows."

num = lambda s: pd.to_numeric(s, errors="coerce")

# --- required columns
abs_main = ["multiple_effect","ebitda_effect","multiple_ebitda_combination_effect","fcf_effect"]
abs_sub  = ["sales_effect","margin_effect","sales_margin_combination_effect"]
rel_all  = [f"relative_{c}" for c in abs_main + abs_sub]

rel = df[rel_all].apply(num)
tm  = num(df["times_money"])
tmu = num(df["tm_unlevered"])
ic  = num(df["invested_capital"])

# ---------- (A) TM-scale (levered) check: (Σ abs main)/IC ≈ TM ----------
sum_abs_main = df[abs_main].apply(num).sum(axis=1, min_count=1)
tm_from_abs  = (sum_abs_main / ic).where(ic != 0)
diff_tm = tm - tm_from_abs
tol_tm  = 0.01 * tm.abs()
ok_tm_row  = (diff_tm.abs() <= tol_tm) & np.isfinite(diff_tm) & np.isfinite(tol_tm)
ok_tm_deal = ok_tm_row.groupby(df["deal_id"]).all()
df["qa_tm_diff"] = diff_tm

# ---------- (B) TM_unlevered via relative shares: Σ (relative_i * tmu) ≈ tmu ----------
# produce TM_unlevered-scaled contributions for ALL effects (main + sub)
for c in abs_main + abs_sub:
    df[f"tmu_contrib_{c}"] = rel[f"relative_{c}"] * tmu

tmu_from_rel = df[[f"tmu_contrib_{c}" for c in abs_main]].sum(axis=1, min_count=1)
diff_tmu = tmu - tmu_from_rel
tol_tmu  = 0.01 * tmu.abs()
ok_tmu_row  = (diff_tmu.abs() <= tol_tmu) & np.isfinite(diff_tmu) & np.isfinite(tol_tmu)
ok_tmu_deal = ok_tmu_row.groupby(df["deal_id"]).all()
df["qa_tmu_diff"] = diff_tmu

# ---------- (C) EBITDA sub-breakdown on TM_unlevered scale ----------
tmu_contrib_ebitda = df["tmu_contrib_ebitda_effect"]
tmu_contrib_subsum = df[["tmu_contrib_sales_effect","tmu_contrib_margin_effect","tmu_contrib_sales_margin_combination_effect"]].sum(axis=1, min_count=1)

diff_tmu_ebitda = tmu_contrib_ebitda - tmu_contrib_subsum
# tolerance: ±1% of |tmu_contrib_ebitda| with small floor
tol_tmu_ebitda = (0.01 * tmu_contrib_ebitda.abs()).where(lambda s: s > 1e-9, 1e-9)
ok_tmu_ebitda_row = (diff_tmu_ebitda.abs() <= tol_tmu_ebitda) & np.isfinite(diff_tmu_ebitda) & np.isfinite(tol_tmu_ebitda)
ok_tmu_ebitda_deal = ok_tmu_ebitda_row.groupby(df["deal_id"]).all()
df["qa_tmu_ebitda_sub_diff"] = diff_tmu_ebitda

# ---- persist ----
df.to_csv(TARGET_CSV, index=False)

# ---- summaries (deal-level) ----
def summarize(name, mask):
    ok = int(mask.sum()); out = int((~mask).sum())
    print(f"{name}: deals within ±1% = {ok} | outside ±1% = {out}")

summarize("TM (levered) identity", ok_tm_deal)
summarize("TM_unlevered (sum of tmu_contrib main effects)", ok_tmu_deal)
summarize("EBITDA sub-breakdown on TM_unlevered scale", ok_tmu_ebitda_deal)

print("tmu_contrib_ columns written for:", abs_main + abs_sub)


TM (levered) identity: deals within ±1% = 413 | outside ±1% = 0
TM_unlevered (sum of tmu_contrib main effects): deals within ±1% = 413 | outside ±1% = 0
EBITDA sub-breakdown on TM_unlevered scale: deals within ±1% = 413 | outside ±1% = 0
tmu_contrib_ columns written for: ['multiple_effect', 'ebitda_effect', 'multiple_ebitda_combination_effect', 'fcf_effect', 'sales_effect', 'margin_effect', 'sales_margin_combination_effect']
